<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>

# **Launch Sites Locations Analysis with Folium**

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.

## Objectives

This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.

In [1]:
import folium
import pandas as pd
import requests
from io import StringIO

# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster

# Import folium MousePosition plugin
from folium.plugins import MousePosition

# Import folium DivIcon plugin
from folium.features import DivIcon

Mark all launch sites on a map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates

The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.

In [2]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df = pd.read_csv(StringIO(requests.get(url).text))
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [3]:
# Select the relevant sub-columns: 'Launch Site', 'Lat', 'Long', 'class'
launch_sites_df = spacex_df[['Launch Site', 'Lat', 'Long']].groupby(['Launch Site'], as_index=False).first()
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.

We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [4]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate.

In [5]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )

site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.

Now, let's add a circle for each launch site in data frame `launch_sites`

In [6]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
for index, row in launch_sites_df.iterrows():
    name, lt, ln = row['Launch Site'], row['Lat'], row['Long']
    circle = folium.Circle([lt, ln], radius=1000, color='#d35400', fill=True).add_child(folium.Popup(name))
    marker = folium.map.Marker(
        [lt, ln],
        # Create an icon as a text label
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{name}</b></div>',
            )
        )

    site_map.add_child(circle)
    site_map.add_child(marker)

site_map


*   all launch sites in proximity to the Equator line
*   all launch sites in very close proximity to the coast

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not

In [7]:
spacex_df.tail(10)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
46,43,2017-10-11,22:53:00,F9 FT B1031.2,KSC LC-39A,SES-11 / EchoStar 105,5200.00,GTO,SES EchoStar,Success (drone ship),1,28.573255,-80.646895
47,44,2017-10-30,19:34:00,F9 B4 B1042.1,KSC LC-39A,Koreasat 5A,3500.00,GTO,KT Corporation,Success (drone ship),1,28.573255,-80.646895
48,54,2018-05-11,20:14:00,F9 B5 B1046.1,KSC LC-39A,Bangabandhu-1,3600.00,GTO,Thales-Alenia/BTRC,Success (drone ship),1,28.573255,-80.646895
49,45,2017-12-15,15:36:00,F9 FT B1035.2,CCAFS SLC-40,SpaceX CRS-13,2205.00,LEO (ISS),NASA (CRS),Success (ground pad),1,28.563197,-80.576820
50,47,2018-01-08,1:00:00,F9 B4 B1043.1,CCAFS SLC-40,Zuma,3696.65,LEO,Northrop Grumman,Success (ground pad),1,28.563197,-80.576820
51,48,2018-01-31,21:25:00,F9 FT B1032.2,CCAFS SLC-40,GovSat-1 / SES-16,4230.00,GTO,SES,Controlled (ocean),0,28.563197,-80.576820
52,50,2018-03-06,5:33:00,F9 B4 B1044,CCAFS SLC-40,Hispasat 30W-6 PODSat,6092.00,GTO,Hispasat NovaWurks,No attempt,0,28.563197,-80.576820
53,52,2018-04-02,20:30:00,F9 B4 B1039.2,CCAFS SLC-40,SpaceX CRS-14,2647.00,LEO (ISS),NASA (CRS),No attempt,0,28.563197,-80.576820
54,53,2018-04-18,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362.00,HEO,NASA (LSP),Success (drone ship),1,28.563197,-80.576820
55,56,2018-06-04,4:45:00,F9 B4 B1040.2,CCAFS SLC-40,SES-12,5384.00,GTO,SES,No attempt,0,28.563197,-80.576820


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.

Let's first create a `MarkerCluster` object

In [8]:
marker_cluster = MarkerCluster()

Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value

In [9]:
def color(x):
    if x == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(color)
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long,marker_color
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356,red
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356,red
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356,red
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356,red
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356,red


For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`

In [10]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# For each row in spacex_df data frame, create a Marker object
# with its coordinates and customize the Marker's icon property
for index, row in spacex_df.iterrows():
    name, lt, ln, clr = row['Launch Site'], row['Lat'], row['Long'], row['marker_color']

    # Create and add a Marker  with customized icon color
    marker = folium.Marker(
        [lt, ln],
        icon=folium.Icon(color=clr, icon_color=clr, icon='launch')
    )
    marker_cluster.add_child(marker)

site_map

From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.

Calculate the distances between a launch site to its proximities

Next, we need to explore and analyze the proximities of launch sites.

Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)

In [11]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

In [12]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

Mark down a point on the coastline using MousePosition and calculate the distance between the coastline point and the launch site.

In [13]:
# Choose the coordinates of the coastline, near Jacksonville
coastline_lat, coastline_lon = 30.31016, -81.39169
launch_site_lat, launch_site_lon = 28.56367, -80.57163

# Calculate the center point between launch site and coastline
center_lat = (launch_site_lat + coastline_lat) / 2
center_lon = (launch_site_lon + coastline_lon) / 2

# Initialize the map with the center point and zoom level
site_map = folium.Map(location=[center_lat, center_lon], zoom_start=8)

# Calculate the distance to the coastline
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create and add a folium.Marker on your selected coastline point on the map
distance_marker = folium.Marker(
   [coastline_lat, coastline_lon],
   icon=DivIcon(
       icon_size=(20, 20),
       icon_anchor=(0, 0),
       html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % "{:.2f} KM".format(distance_coastline)
   )
)

# Draw a PolyLine between the launch site and the selected coastline point
# Create a folium.PolyLine object using the coastline and launch site coordinates
lines = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]], weight=1)

# Add the marker and polyline to the map
site_map.add_child(distance_marker)
site_map.add_child(lines)

# Display the map
site_map


Similarly, let's draw a line between a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first

A railway map symbol may look like this:
<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>

In [14]:
# Find the coordinates of the railway
railway_lat, railway_lon = 28.54725, -80.79715
launch_site_lat, launch_site_lon = 28.56367, -80.57163

# Calculate the center point between launch site and railway
center_lat = (launch_site_lat + railway_lat) / 2
center_lon = (launch_site_lon + railway_lon) / 2

# Initialize the map with the center point and zoom level
site_map = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Calculate the distance to the railway
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)

# Create and add a folium.Marker on your selected closest railway point on the map
distance_marker = folium.Marker(
   [railway_lat, railway_lon],
   icon=DivIcon(
       icon_size=(20, 20),
       icon_anchor=(0, 0),
       html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % "{:.2f} KM".format(distance_railway)
   )
)

# Draw a PolyLine between the launch site and the selected railway point
# Create a folium.PolyLine object using the railway and launch site coordinates
lines = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [railway_lat, railway_lon]], weight=1)

# Add the marker and polyline to the map
site_map.add_child(distance_marker)
site_map.add_child(lines)

# Display the map
site_map


A highway map symbol may look like this:
<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>



In [15]:
# Find the coordinates of the highway
highway_lat, highway_lon = 28.45164, -81.17935
launch_site_lat, launch_site_lon = 28.56367, -80.57163

# Calculate the center point between launch site and highway
center_lat = (launch_site_lat + highway_lat) / 2
center_lon = (launch_site_lon + highway_lon) / 2

# Initialize the map with the center point and zoom level
site_map = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Calculate the distance to the highway
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

# Create and add a folium.Marker on your selected closest highway point on the map
distance_marker = folium.Marker(
   [highway_lat, highway_lon],
   icon=DivIcon(
       icon_size=(20, 20),
       icon_anchor=(0, 0),
       html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % "{:.2f} KM".format(distance_highway)
   )
)

# Draw a PolyLine between the launch site and the selected highway point
# Create a folium.PolyLine object using the highway and launch site coordinates
lines = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [highway_lat, highway_lon]], weight=1)

# Add the marker and polyline to the map
site_map.add_child(distance_marker)
site_map.add_child(lines)

# Display the map
site_map


A city map symbol may look like this:
<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>



In [16]:
# Find the coordinates of the city Orlando
city_lat, city_lon = 28.53808, -81.38672
launch_site_lat, launch_site_lon = 28.56367, -80.57163

# Calculate the center point between launch site and Orlando city
center_lat = (launch_site_lat + city_lat) / 2
center_lon = (launch_site_lon + city_lon) / 2

# Initialize the map with the center point and zoom level
site_map = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Calculate the distance to the city Orlando
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

# Create and add a folium.Marker on your selected closest city point on the map
distance_marker = folium.Marker(
   [city_lat, city_lon],
   icon=DivIcon(
       icon_size=(20, 20),
       icon_anchor=(0, 0),
       html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % "{:.2f} KM".format(distance_city)
   )
)

# Draw a PolyLine between the launch site and the selected city point
# Create a folium.PolyLine object using the city and launch site coordinates
lines = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [city_lat, city_lon]], weight=1)

# Add the marker and polyline to the map
site_map.add_child(distance_marker)
site_map.add_child(lines)

# Display the map
site_map
